In [45]:
from webdriver_manager.chrome import ChromeDriverManager
import csv
import os
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from urllib.parse import urljoin
import requests
import os
import traceback
from icecream import ic
import base64


def remove_claim_button(driver):
    remove_script = """
    var element = document.evaluate(arguments[0], document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {
        element.parentNode.removeChild(element);
    }
    """

    # XPath of the element to be removed
    xpath_to_remove = '//*[@id="teamfresh"]/div/div/div[1]'

    # Execute the script
    driver.execute_script(remove_script, xpath_to_remove)


# get_all_sms 함수화
#
def get_all_sms(driver):
    extract_sms = True
    try:
        # Click on the first element
        first_element_xpath = (
            '//*[@id="teamfresh"]/div/div/div[2]/div[2]/div/table[1]/tbody/tr/td[2]/div'
        )
        first_element = driver.find_element(By.XPATH, first_element_xpath)
        first_element.click()
        time.sleep(0.5)
    except NoSuchElementException:
        traceback.print_exc()
        print("Element not found. SMS 추출 취소")
        extract_sms = False

    if extract_sms:
        # Wait for the new element to appear and get its text
        new_element_xpath = "/html/body/div[2]/div/div[1]/div/div/div[2]/p"
        new_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, new_element_xpath))
        )
        ic(new_element)
        ic(new_element.text)
        tmp_text = driver.execute_script("return arguments[0].innerHTML;", new_element)
        ic(tmp_text)
        new_text = new_element.text

        # close the popup
        close_xpath = "/html/body/div[2]/div/div[1]/div/div/div[1]/button"
        close_bttn = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, close_xpath))
        )
        close_bttn.click()

        # Set the new text to the target element
        target_element_xpath = (
            '//*[@id="teamfresh"]/div/div/div[2]/div[2]/div/table[1]/tbody/tr/td[2]'
        )
        target_element = driver.find_element(By.XPATH, target_element_xpath)
        driver.execute_script(
            "arguments[0].innerText = arguments[1];", target_element, new_text
        )


#
# get_image 함수화
#
def get_image(driver):
    current_url = driver.current_url
    try:
        # Check and click the specified element
        icon_xpath = (
            '//*[@id="teamfresh"]/div/div/div[2]/div[2]/div/table[1]/tbody/tr/td[3]/i'
            if "shipping" in current_url
            else '//*[@id="teamfresh"]/div/div/div/div[2]/div/table[1]/tbody/tr/td[3]/i'
        )
        icon_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, icon_xpath))
        )
        icon_element.click()
        # Wait for the new image element and get its 'src' attribute
        img_xpath = '//*[@id="imgTag"]'
        img_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, img_xpath))
        )
        blob_img_url = img_element.get_attribute("src")
        image_bytes = get_file_content_chrome(driver, blob_img_url)

        # Download the image and save it locally
        img_folder = "img"
        os.makedirs(img_folder, exist_ok=True)
        img_name = blob_img_url.split("/")[-1] + ".jpg"
        img_path = os.path.join(img_folder, img_name)
        with open(img_path, "wb") as img_file:
            img_file.write(image_bytes)

        time.sleep(1)
        # Click the button to return to the previous screen
        back_button_xpath = (
            "/html/body/div[2]/div/div[1]/div/div/div[1]/button"
            if "shipping" in current_url
            else "/html/body/div[2]/div/div[1]/div/div/div[1]/h5/div/button"
        )
        back_button = driver.find_element(By.XPATH, back_button_xpath)
        back_button.click()
        time.sleep(0.5)

        # Replace the original icon element with the downloaded image
        driver.execute_script(
            """
            var iconElement = arguments[0];
            var newImg = document.createElement('img');
            newImg.src = arguments[1];
            iconElement.parentNode.replaceChild(newImg, iconElement);
            """,
            icon_element,
            img_path,
        )
        time.sleep(0.5)
    except NoSuchElementException:
        traceback.print_exc()
        print("Element not found. 이미지 추출 취소")
        # ESC 키를 보내어 팝업을 무시
        WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        ).send_keys(Keys.ESCAPE)


def select_second_tab(driver):
    # 두 번째 탭으로 이동
    element_to_click = driver.find_element(
        By.XPATH, '//*[@id="teamfresh"]/div/div/ul/li[2]/a'
    )
    element_to_click.click()


def embed_js_for_tabselection(driver):
    # 페이지의 HTML 내용 가져오기
    html_content = driver.page_source

    # 탭선택 가능하게 하는 JavaScript snippet
    javascript_snippet = """
    <script>
    document.addEventListener('DOMContentLoaded', function() {
        var tabs = document.querySelectorAll('.nav-item .nav-link');
        var tabPanes = document.querySelectorAll('.tab-content .tab-pane');
        tabs.forEach(function(tab, index) {
            tab.addEventListener('click', function(event) {
                event.preventDefault();
                tabs.forEach(t => t.classList.remove('active'));
                tabPanes.forEach(pane => pane.classList.remove('active'));
                tab.classList.add('active');
                tabPanes[index].classList.add('active');
            });
        });
    });
    </script>
    """
    # Insert the JavaScript snippet before the closing </body> tag
    html_content = html_content.replace("</body>", javascript_snippet + "</body>")
    return html_content


def get_css_and_link_to_local(driver, html_content):
    # 'css' 및 'js' 폴더 생성
    css_folder = "css"
    os.makedirs(css_folder, exist_ok=True)

    # 기본 경로 설정
    base_path = "./"  # 웹 서버에서의 경로

    # CSS 및 JavaScript 파일 처리
    for tag in ["link"]:
        elements = driver.find_elements(By.TAG_NAME, tag)
        for element in elements:
            href = (
                element.get_attribute("href")
                if tag == "link"
                else element.get_attribute("src")
            )

            if href and (
                (tag == "link" and "stylesheet" in element.get_attribute("rel"))
                or tag == "script"
            ):
                # 절대 URL로 변환
                # old_href = href
                # ic(old_href)

                href = urljoin(remote_url, href)
                file_name = href.split("/")[-1]
                file_path = os.path.join(css_folder, file_name)

                # 파일 다운로드 및 저장
                response = requests.get(href)
                with open(file_path, "wb") as file:
                    file.write(response.content)

                # HTML 내용에서 링크 경로 수정
                new_href = base_path + os.path.join(css_folder, file_name)
                old_href = "/static/" + os.path.join(css_folder, file_name)
                # ic(old_href)
                # ic(new_href)
                html_content = html_content.replace(old_href, new_href)
    return html_content


def get_file_content_chrome(driver, uri):
    result = driver.execute_async_script(
        """
        var uri = arguments[0];
        var callback = arguments[1];
        var xhr = new XMLHttpRequest();
        xhr.responseType = 'arraybuffer';
        xhr.onload = function() {
            var reader = new FileReader();
            reader.readAsDataURL(new Blob([xhr.response]));
            reader.onloadend = function() {
                callback(reader.result);
            }
        };
        xhr.onerror = function() { callback(xhr.status) };
        xhr.open('GET', uri);
        xhr.send();
    """,
        uri,
    )
    if isinstance(result, int):
        raise Exception("Request failed with status %s" % result)
    return base64.b64decode(result.split(",")[1])


def navigate_to_page(url, wait_xpath, sec_to_wait=10):
    try:
        driver.get(url)
        WebDriverWait(driver, sec_to_wait).until(
            EC.presence_of_element_located((By.XPATH, wait_xpath))
        )
        time.sleep(sec_to_wait)
    except Exception as e:
        print(f"Error navigating to {url}: {e}")


def open_and_login(driver):
    # 페이지 로드
    driver.get(remote_url)
    time.sleep(1)

    # 로그인 페이지 확인 및 로그인 수행
    try:
        # ID와 Password 입력 필드를 찾아 값을 입력
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "userLoginId"))
        )
        id_field = driver.find_element(By.ID, "userLoginId")
        password_field = driver.find_element(By.ID, "userLoginPw")

        id_field.send_keys("dummyid")
        password_field.send_keys("dummypassword")

        # 로그인 버튼 클릭
        login_button = driver.find_element(
            By.XPATH, '//*[@id="teamfresh"]/div/div/form/button'
        )
        login_button.click()
    except Exception as e:
        print("로그인 페이지를 찾을 수 없습니다:", e)





profile_path = "~/.config/google-chrome/Profile 9"
chrome_options = Options()
chrome_options.add_argument("--user-data-dir=" + profile_path)
chrome_service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
# 원격 페이지 URL
remote_url = "https://tms.teamfresh.co.kr"

open_and_login(driver)
time.sleep(1)

# shipping 배송 상세 페이지 로드
detail_page_url = "https://tms.teamfresh.co.kr/popup/shipping/report/25309317"
navigate_to_page(
    detail_page_url,
    '//*[@id="teamfresh"]/div/div/ul/li[1]/a/span',
    3,
)

# # return 반품 상세 페이지 로드
# detail_page_url = "https://tms.teamfresh.co.kr/popup/return/report/25331748"
# navigate_to_page(
#     detail_page_url,
#     '//*[@id="teamfresh"]/div/div/ul/li[1]/a',
#     3,
# )


select_second_tab(driver)
time.sleep(1)
get_all_sms(driver)
get_image(driver)
if "shipping" in driver.current_url:
    remove_claim_button(driver)  # 배송 상세 페이지의 배상버튼 제거
html_content = embed_js_for_tabselection(driver)
html_content = get_css_and_link_to_local(driver, html_content)

# 수정된 HTML 내용을 파일로 저장
with open("tmp_detail.html", "w", encoding="utf-8") as file:
    file.write(html_content)


# 작업 완료 후 드라이버 종료
driver.quit()

ic| new_element: <selenium.webdriver.remote.webelement.WebElement (session="e02f2ba98a97f22ad181d68a8921a4f2", element="F6F9D3E66E010433DE4FFC5A676F27EC_element_41")>
ic| new_element.text: ('[배송완료] 새벽을 여는 설레임 새벽배송 No.1 팀프레시입니다. 하루 시작 전 한윤혜고객님의 소중한 상품을 바로 받아보세요 ! ▶발송인 '
                       ': 클린베딩 ▶배송위치 : 현관 문 앞 ▶송장번호 : T01034060000113A01 배송문의 : 1660-1107 배송문의 운영시간 '
                       ': 7시~16시 상품문의 : 070-4757-2111 상품문의 운영시간 : 09:00~18:00')
ic| tmp_text: ('[배송완료] 새벽을 여는 설레임
              '
               '
              '
               '새벽배송 No.1 팀프레시입니다. 
              '
               '하루 시작 전 한윤혜고객님의 소중한 상품을 바로 받아보세요 !
              '
               '
              '
               '▶발송인 : 클린베딩
              '
               '▶배송위치 : 현관 문 앞
              '
               '▶송장번호 : T01034060000113A01
              '
               '배송문의 : 1660-1107 
              '
               '배송문의 운영시간 : 7시~16시
              '
               '상품문의 : 070-4757-2111
              '
         